In [16]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
from splinter import Browser
import os
import numpy as np

In [13]:
# URL of page to be scraped
#url = 'https://www.walgreens.com/store/c/halloween-candy/ID=520931-tier3'
url = 'https://www.walgreens.com/store/store/category/productlist.jsp?N=520931&Eon=520931'


In [17]:
# Read HTML from file
filepath = os.path.join("Resources", "walgreens.html")
with open(filepath) as file:
    wal_html = file.read()

In [18]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [19]:
html = browser.html
soup = BeautifulSoup(wal_html, 'html.parser')

In [20]:
# <div class="brand" id="prodBrandNamecompare_sku6253945" data-reactid="468">PayDay</div>
# "brand"
#<input type="checkbox" id="filter_353669" tabindex="0" value="353669" name="353669" title="Sour Punch(2)">
results = soup.find_all("div", class_="item card card__product")
print(results)

t store</span></div><div class="avail-cta" id="not-available-for-shippingcompare_sku6151899" name="not-available-for-shipping compare_sku6151899"><!-- react-text: 4398 -->Not available for shipping<!-- /react-text --><span class="sr-only">Not available for shipping</span></div><button class="btn btn__blue" id="pickup-ship-btncompare_sku6151899" name="pickup-ship-btn"><!-- react-text: 4401 -->Pick up<!-- /react-text --><span class="sr-only">will open overlay</span></button></div></div><span aria-live="polite" class="sr-only" id="ownbrandAnnounce"></span></div>, <div class="item card card__product" id="productcard300400854"><div aria-hidden="true" id="productcardheight300400854" style="text-align: center;"><a href="/store/c/haribo-sweet-%26-scary-mix-assorted,-laydown-bag/ID=300400854-product" id="productOmniSelectcompare_400622247" tabindex="-1"><figure class="product__img"></figure><span class="sr-only"><!-- react-text: 4410 -->Haribo Sweet &amp; Scary Mix Assorted, Laydown Bag<!-- /re

In [21]:
len(results)

72

In [23]:
ratings_list = []

for result in results:
    candy_ratings = {}
    candy_ratings['brand'] = result.find('div', class_='brand').text
    candy_ratings['description'] = result.find('strong', class_='description').text
    try:
        candy_ratings['rating'] = result.find("span", class_="product__rating").img["alt"]
        
    except AttributeError:
        candy_ratings['rating'] = np.nan
    ratings_list.append(candy_ratings)
ratings_list

[{'brand': 'PayDay',
  'description': 'Snack Size Peanut Caramel Bars Peanut Caramel',
  'rating': '4.6 out of 5 total 12 reviews'},
 {'brand': 'Snickers',
  'description': 'Peanut Butter Squared Fun Size Candy',
  'rating': '5.0 out of 5 total 1 reviews'},
 {'brand': 'Tootsie Roll',
  'description': 'Caramel Apple Pops Green Apple',
  'rating': '5.0 out of 5 total 6 reviews'},
 {'brand': 'Milky Way',
  'description': 'Chocolate Candy Bar Fun Size',
  'rating': '4.9 out of 5 total 44 reviews'},
 {'brand': "Reese's",
  'description': 'Snack Size Peanut Butter Cups',
  'rating': '5.0 out of 5 total 3 reviews'},
 {'brand': 'Mounds',
  'description': 'Snack Size Candy Bars Dark Chocolate Coconut Filled',
  'rating': '4.8 out of 5 total 16 reviews'},
 {'brand': "Hershey's",
  'description': 'All Time Greats Snack Size Assortment, 30 Pieces',
  'rating': '3.9 out of 5 total 37 reviews'},
 {'brand': 'Kisses',
  'description': 'Gold with Pretzel Bits Candy',
  'rating': '5.0 out of 5 total 1 r

In [31]:
ratings = soup.find_all("span", class_="product__rating")
print(ratings)

[<span class="product__rating" data-reactid="488"><figure data-reactid="489"><img alt="4.6 out of 5 total 12 reviews" data-reactid="490" id="reviewbazarcompare_sku6253945" src="//www.walgreens.com/images/adaptive/bvrating/rating-4_6.gif" title="4.6 out of 5"/><figcaption data-reactid="491">12</figcaption></figure></span>, <span class="product__rating" data-reactid="542"><figure data-reactid="543"><img alt="5.0 out of 5 total 1 reviews" data-reactid="544" id="reviewbazarcompare_sku6062317" src="//www.walgreens.com/images/adaptive/bvrating/rating-5_0.gif" title="5.0 out of 5"/><figcaption data-reactid="545">1</figcaption></figure></span>, <span class="product__rating" data-reactid="591"><figure data-reactid="592"><img alt="5.0 out of 5 total 6 reviews" data-reactid="593" id="reviewbazarcompare_sku6180836" src="//www.walgreens.com/images/adaptive/bvrating/rating-5_0.gif" title="5.0 out of 5"/><figcaption data-reactid="594">6</figcaption></figure></span>, <span class="product__rating" data

In [34]:
df = pd.DataFrame(ratings_list)
df = df.dropna(how='any')
df.head()

,brand,description,rating
0,PayDay,Snack Size Peanut Caramel Bars Peanut Caramel,4.6 out of 5 total 12 reviews
1,Snickers,Peanut Butter Squared Fun Size Candy,5.0 out of 5 total 1 reviews
2,Tootsie Roll,Caramel Apple Pops Green Apple,5.0 out of 5 total 6 reviews
3,Milky Way,Chocolate Candy Bar Fun Size,4.9 out of 5 total 44 reviews
4,Reese's,Snack Size Peanut Butter Cups,5.0 out of 5 total 3 reviews


In [37]:

for index, row in df.iterrows():
    # extract the individual rating
    rating = row['rating'][0:3]
    df.loc[index, 'individual_rating'] = float(rating)
    # extract number of reviews    
    count = [int(i) for i in row['rating'].split() if i.isnumeric()]
    df.loc[index, 'review'] = count[1]
df

,brand,description,rating,individual_rating,review
0,PayDay,Snack Size Peanut Caramel Bars Peanut Caramel,4.6 out of 5 total 12 reviews,4.6,12
1,Snickers,Peanut Butter Squared Fun Size Candy,5.0 out of 5 total 1 reviews,5.0,1
2,Tootsie Roll,Caramel Apple Pops Green Apple,5.0 out of 5 total 6 reviews,5.0,6
3,Milky Way,Chocolate Candy Bar Fun Size,4.9 out of 5 total 44 reviews,4.9,44
4,Reese's,Snack Size Peanut Butter Cups,5.0 out of 5 total 3 reviews,5.0,3
5,Mounds,Snack Size Candy Bars Dark Chocolate Coconut F...,4.8 out of 5 total 16 reviews,4.8,16
6,Hershey's,"All Time Greats Snack Size Assortment, 30 Pieces",3.9 out of 5 total 37 reviews,3.9,37
7,Kisses,Gold with Pretzel Bits Candy,5.0 out of 5 total 1 reviews,5.0,1
8,Snickers,Chocolate Candy Bars Fun Size,4.7 out of 5 total 27 reviews,4.7,27
9,Hershey's,Snack Size Milk Chocolate Bars Milk Chocolate,4.7 out of 5 total 44 reviews,4.7,44


https://towardsdatascience.com/elevate-your-webscraping-with-splinter-a926eee7f7d9